# 爬取豆瓣Top250电影短评

爬虫目标：爬取豆瓣Top250电影短评数据<br>
由于要爬取的页面太多，容易被封号，所以采取了三个反爬方法:延长请求间隔时间、更换IP和伪造cookies，采取这三个方法后没有那么容易被封号。

爬虫过程：
* 爬取代理IP
* 伪造cookies
* 模拟登陆
* 爬取页面内容解析评论并写入文件

In [1]:
import requests
import time
from lxml import etree
import random

In [14]:
def get_page(url, headers, page_num, step):
    """
    Desc: 用于获取网页内容
    Params:
        url: 首页网址
        headers: 请求头
        page_num: 所需获取网页最后一页的页码，通过循环用来组合除首页外其它页的网址
    Return: 
        html: 每个网页的内容
    """
    html = ''
    for i in range(1, page_num, step):
        page = url + str(i) # 构造每页网址
        r = requests.get(page, headers=headers) # 提交请求
        print (r.status_code) # 打印每页请求的状态码
        r.encoding = r.apparent_encoding # 设置编码方式，避免出现乱码
        html += r.text
        time.sleep(3)
    return html

## 抓取代理IP

### 解析每页IP相关信息

In [3]:
def get_ip(html):
    """
    Desc: 从网页内容中解析出IP, 端口, IP类型
    Params:
        html: 网页内容
    Return:
        ips: 列表, 包含解析得到的所有IP地址
        ports: 列表, 包含解析得到的所有IP地址对应的端口信息
        types: 列表, 包含解析得到的所有IP地址对应的IP类型信息
    """
    # 初始化三个列表
    ips = []
    ports = []
    types = []
    content = etree.HTML(html)
    for item in content.xpath('//tr[@class="odd"]'):
        if ('分钟' not in item): # 爬取存活时间一小时以上的IP
            if item.xpath('./td[6]/text()')[0] == 'HTTPS': # 爬取IP类型为HTTPS的代理IP
                ips.append(item.xpath('./td[2]/text()')[0]) # 解析IP
                ports.append(item.xpath('./td[3]/text()')[0]) # 解析IP端口信息
                types.append(item.xpath('./td[6]/text()')[0]) # 解析IP类型
    return ips, ports, types

### 验证抓取到的IP是否有效

In [4]:
def verif_ip(ips, ports, types, url, headers):
    """
    Desc: 验证获取IP地址的有效性, 并将有效IP地址写入文件
    Params:
        ips: 列表, 包含所有IP地址
        ports: 列表, 包含所有IP地址对应的端口信息
        types: 列表, 包含所有IP地址对应的IP类型信息
        url: 验证IP有效性的网址
        headers: 请求头
    """
    for ip, port, iptype in zip(ips, ports, types):
        proxy = '%s://%s:%s' % (iptype, ip, port) # 将解析得到的IP相关信息组合
        # 用该IP访问网页验证IP是否有效
        s = requests.get(url=url, headers=headers, proxies={iptype:proxy})
        if s.status_code == 200: # 访问成功为有效IP
            if s.text:
                print ('有效IP：', proxy)
                # 将有效IP写入文件
                with open('proxies.csv', 'a') as fd:
                    fd.write('%s:%s' % (iptype,proxy))
                    fd.write('\n')
            else:
                print ('无效IP: ', proxy)
        else:
            print ('获取页面失败: ', s.status_code)
        time.sleep(3)

In [6]:
def get_proxies(file):
    """
    Desc: 读取有效IP文件中的内容
    Params:
        file: 有效IP文件名
    Return:
        proxies: 字典, 每一个元素key为IP类型, value为IP
    """
    proxies = []
    f = open(file, 'r')
    for row in f.readlines(): # 读入文件每一行
        row = row[:-1] # 去掉换行符
        proxies.append({row.split(':', 1)[0]:row.split(':', 1)[1]})
    return proxies

In [5]:
header = {'Accept': '*/*',
               'Accept-Language': 'en-US,en;q=0.8',
               'Cache-Control': 'max-age=0',
               'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/48.0.2564.116 Safari/537.36',
               'Connection': 'keep-alive'}
# 爬取西刺网国内高匿代理网页内容
html = get_page(url='http://www.xicidaili.com/nn/', headers=header, page_num=20, step=1)
# 解析网页中的IP、端口信息、IP类型
ips, ports, types = get_ip(html)
# 将有效IP写入文件
verif_ip(ips, ports, types, url='http://www.baidu.com/', headers=header)
# 读取文件中所有的IP
proxies = get_proxies('proxies.csv')

200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
有效IP： HTTPS://117.81.59.246:38492
有效IP： HTTPS://114.231.115.209:46322
有效IP： HTTPS://117.36.103.170:8118
有效IP： HTTPS://117.95.199.86:44606
有效IP： HTTPS://222.85.39.162:808
有效IP： HTTPS://117.93.1.101:40268
有效IP： HTTPS://115.230.76.176:31246
有效IP： HTTPS://180.119.65.112:808
有效IP： HTTPS://113.128.28.39:48452
有效IP： HTTPS://49.83.24.179:808
有效IP： HTTPS://61.143.17.235:40714
有效IP： HTTPS://218.20.55.63:8118
有效IP： HTTPS://113.105.203.221:3128
有效IP： HTTPS://115.203.205.156:35039
有效IP： HTTPS://221.229.18.103:3128
有效IP： HTTPS://60.184.172.97:3128
有效IP： HTTPS://1.199.192.23:42274
有效IP： HTTPS://110.73.32.152:8123
有效IP： HTTPS://222.85.50.87:808
有效IP： HTTPS://36.25.24.108:29758
有效IP： HTTPS://123.114.57.83:8118
有效IP： HTTPS://140.250.159.189:33623
有效IP： HTTPS://115.225.159.167:3128
有效IP： HTTPS://60.184.207.115:40224
有效IP： HTTPS://110.72.45.98:8123
有效IP： HTTPS://122.4.42.64:40044
有效IP： HTTPS://114.226.166.118:34993
有效IP： HTTPS://1

有效IP： HTTPS://119.183.130.90:8118
有效IP： HTTPS://110.73.49.193:8123
有效IP： HTTPS://218.14.140.201:41268
有效IP： HTTPS://180.118.241.61:808
有效IP： HTTPS://121.228.99.235:8118
有效IP： HTTPS://124.228.239.114:3128
有效IP： HTTPS://110.73.32.255:8123
有效IP： HTTPS://113.93.100.233:49352
有效IP： HTTPS://182.202.154.88:8080
有效IP： HTTPS://123.54.226.153:38176
有效IP： HTTPS://183.71.136.98:8118
有效IP： HTTPS://121.206.140.161:49704
有效IP： HTTPS://1.196.135.109:38882
有效IP： HTTPS://121.31.102.84:8123
有效IP： HTTPS://49.85.2.58:29911
有效IP： HTTPS://140.250.158.69:49628
有效IP： HTTPS://220.162.155.104:49243
有效IP： HTTPS://59.51.123.212:3128
有效IP： HTTPS://1.199.194.75:39978
有效IP： HTTPS://113.86.220.3:808
有效IP： HTTPS://110.73.43.197:8123
有效IP： HTTPS://115.221.112.55:33731
有效IP： HTTPS://121.60.85.45:8118
有效IP： HTTPS://27.40.146.33:61234
有效IP： HTTPS://171.14.91.119:21186
有效IP： HTTPS://121.31.100.229:8123
有效IP： HTTPS://171.39.42.164:8123
有效IP： HTTPS://125.126.168.241:33808
有效IP： HTTPS://118.75.195.136:8118
有效IP： HTTPS://106.56

有效IP： HTTPS://110.88.127.28:44048
有效IP： HTTPS://27.190.25.136:8118
有效IP： HTTPS://1.196.158.204:32226
有效IP： HTTPS://182.34.16.90:28837
有效IP： HTTPS://120.40.135.116:25163
有效IP： HTTPS://180.106.251.183:8118
有效IP： HTTPS://27.40.148.162:61234
有效IP： HTTPS://121.231.147.12:6666
有效IP： HTTPS://49.71.81.12:3128
有效IP： HTTPS://113.222.83.12:808
有效IP： HTTPS://60.182.37.209:8118
有效IP： HTTPS://119.164.107.98:8118
有效IP： HTTPS://121.31.177.251:8123
有效IP： HTTPS://122.242.89.84:33692
有效IP： HTTPS://49.71.81.200:3128
有效IP： HTTPS://27.40.145.143:61234
有效IP： HTTPS://101.229.165.22:8118
有效IP： HTTPS://113.121.42.35:41580
有效IP： HTTPS://61.143.17.215:40130
有效IP： HTTPS://153.34.126.119:8118
有效IP： HTTPS://175.16.120.122:80
有效IP： HTTPS://183.52.104.132:42362
有效IP： HTTPS://49.71.81.155:3128
有效IP： HTTPS://115.193.99.212:61234
有效IP： HTTPS://110.82.103.157:48753
有效IP： HTTPS://124.231.64.151:3128
有效IP： HTTPS://121.31.151.38:8123
有效IP： HTTPS://59.58.242.236:36052
有效IP： HTTPS://183.151.41.198:3128
有效IP： HTTPS://115.46.97.

## 伪造cookies

一开始没有并没有伪造cookies值，很快就被封号，后面尝试了登录后取出cookies值用来模拟登陆，也很快就被封号了(PS: 可能是我方法不对)，去谷歌发现有人提到可以伪造cookies值，观察自己访问豆瓣的cookies变化和做了一些小小的实验，发现ck、ll和bid三个值可以自己伪造。<br>
这里主要伪造三种cookies值：ck、ll和bid。<br>
* ck：随机抽样4个数字组合
* ll：随机抽样5个或6个数字组合
* bid：从大小写字母和数字随机抽取8、9、10、11个进行组合

In [8]:
def make_cookies():
    """
    Desc: 伪造cookies值, 防止被封得那么快
    Return:
        cookies: 列表, 所有伪造的cookies
    """
    
    letters = [chr(i) for i in range(65, 91)] + [chr(i) for i in range(97, 123)] # 创建包含26个字母大小写的列表
    nums = [i for i in range(0, 10)] # 创建数字0-9的列表

    # 构造600个ck的值
    ck_values = []
    for i in range(0, 600):
        # 从数字列表中随机抽样4个数字后组合在一起
        ck_values.append(''.join(random.sample(letters, 4))) 
    ck_values = set(ck_values) # 去重

    # 构造600个ll的值
    ll_values = []
    for i in range(0, 600):
        # 随机要抽取数字的个数
        num = random.sample([5, 6], 1)[0]
        # 从数字列表中随机抽样5个或6个数字后组合在一起
        ll_values.append(''.join([str(x) for x in random.sample(nums, num)]))
    ll_values = set(ll_values) # 去重
    
    bid_values = []
    all_elements = letters + nums # 创建同时包含26个字母大小写和数字0-9的列表
    # 构造600个bid的值
    for i in range(0, 600):
        # 随机要抽取字符的个数
        num = random.sample([8, 9, 10, 11], 1)[0]
        # 从字符列表中随机抽样一定个数字符后组合在一起
        bid_values.append(''.join([str(x) for x in random.sample(all_elements, num)]))
    bid_values = set(bid_values) # 去重
    
    # 将每个种类伪造的cookies值与相应种类组合成列表后添加到列表cookies中
    cookies = []
    for ck_value in ck_values:
        cookies.append(['ck', ck_value])
    for ll_value in ll_values:
        cookies.append(['ll', ll_value])
    for bid_value in bid_values:
        cookies.append(['bid', bid_value])
    return cookies

In [9]:
cookies = make_cookies()

## 获取电影评论页面地址

因为豆瓣对短评数据显示有限制，登录后也只能爬取500条短评，为了爬取更多短评，看了不同类型短评页面的地址后，将不同类型短评地址中相同的后半部分和电影的地址组合得到每部电影不同类型短评的首页地址。

In [11]:
def get_film_urls(html):
    """
    Desc: 从索引页内容获取每部电影的网址
    Params: 
        html: 网页内容
    Return:
        film_urls: 列表, Top250所有电影的网址
    """
    film_urls = []
    page_content = etree.HTML(html)
    for item in page_content.xpath('//div[@class="hd"]'):
        film_urls.append(item.xpath('./a/@href')[0]) # 获取每部电影地址
    return film_urls

In [12]:
def get_comment_urls(film_urls):
    """
    Desc: 构造每部电影好评、中评、差评和最新评论的首页
    Params:
        film_urls: 列表, 所有电影的网址
    Return:
        comment_urls: 列表, 所有电影好评、中评、差评和最新评论的首页地址
    """
    high = 'comments?status=P&percent_type=h' # 电影好评地址的后半部分
    middle = 'comments?status=P&percent_type=m' # 电影中评地址的后半部分
    low = 'comments?status=P&percent_type=l' # 电影差评地址的后半部分
    latest = 'comments?sort=time&status=P' # 电影最新评论地址的后半部分

    # 将每部电影的地址和每种评论的地址后半部分组合起来
    comment_urls = []
    for film_url in film_urls:
        for comment_type in [high, middle, low, latest]:
            comment_urls.append(film_url + comment_type)
    return comment_urls

In [16]:
# 获取电影索引页内容
film_pages = get_page(url='https://movie.douban.com/top250?start=',
                     headers=header,
                     page_num=250,
                     step=25)
# 解析电影地址
film_urls = get_film_urls(film_pages)
# 构建每部电影对应的好评、中评、差评和最新评论的地址
comment_urls = get_comment_urls(film_urls)
with open('comment_urls.csv', 'a') as f:
    for comment_url in comment_urls:
        f.write(comment_url+'\n')

200
200
200
200
200
200
200
200
200
200


## 模拟登陆

In [36]:
# 表单
formdata = {'form_email':'benynat6@gmail.com',
            'form_password':'894635czx',
            'login':'登录',
            'source':'movie'}
login_header = {
    'User-Agent':'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.11 (KHTML, like Gecko) Chrome/23.0.1271.64 Safari/537.11',
    'Accept':'text/html;q=0.9,*/*;q=0.8',
    'Accept-Charset':'ISO-8859-1,utf-8;q=0.7,*;q=0.3',
    'Accept-Encoding':'gzip',
    'Connection':'keep-alive',
    'Referer':'https://movie.douban.com/top250'}
s = requests.session()
# 获取登录页面
login = s.get('https://www.douban.com/accounts/login?source=movie', headers=login_header, verify=False)
login_page = etree.HTML(login.text)
# 如果有验证码，处理验证码
if login_page.xpath('//img[@id="captcha_image"]'):
    captcha_url = login_page.xpath('//img[@id="captcha_image"]/@src')[0] # 解析验证码地址
    print ('验证码链接：')
    print (captcha_url)
    formdata['captcha-solution'] = input('请输入验证码：') # 输入验证码，作为表单信息一部分
    formdata['captcha-id'] = login_page.xpath('//input[@name="captcha-id"]/@value')   
    r = s.post('https://www.douban.com/accounts/login?source=movie', data=formdata) # 提交表单信息并登陆
    print (r.url, r.status_code) # 验证是否登陆成功

F:\python\lib\site-packages\urllib3\connectionpool.py:858: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


验证码链接：
https://www.douban.com/misc/captcha?id=mljctVVhsStBwGP0mPRIBMk0:en&size=s
请输入验证码：process
https://movie.douban.com/top250 200


## 抓取评论页面并解析评论

In [34]:
def get_comments(content):
    """
    Desc: 对短评页面进行解析，得到每条短评的信息——电影名称、用户主页地址、用户名、用户评分、评论时间、短评内容、赞同人的数量
    Params:
        content: 短评页面的内容
    Return:
        comments: 列表，每个元素都由相关短评信息组合得到
    """
    comments = []
    film_name = content.xpath('//div[@id="content"]/h1/text()')[0] # 解析电影名字
    for item in content.xpath('//div[@class="comment"]'):
        user_link = item.xpath('./h3/span[@class="comment-info"]/a/@href')[0] # 解析用户页面地址
        # 有些用户已经注销，那么用户名标记为None
        if item.xpath('./h3/span[@class="comment-info"]/a/text()'):
            user_name = item.xpath('./h3/span[@class="comment-info"]/a/text()')[0] # 解析用户名字
        else:
            user_name = None
        user_score = item.xpath('./h3/span[@class="comment-info"]/span[2]/@class')[0][7] # 解析用户评分
        comment_time = item.xpath('./h3/span[@class="comment-info"]/span[@class="comment-time "]/@title')[0] # 解析评论时间
        comment = item.xpath('./p/text()')[0] # 解析短评
        follower = item.xpath('./h3/span[@class="comment-vote"]/span/text()')[0] # 解析赞同人的数量
        # 将每条短评相关信息组合成一条并写入文件
        comments.append('%s,%s,%s,%s,%s,%s,%s' % (film_name, user_link, user_name, user_score,
                                              comment_time, comment, follower))
    return comments

注：因为代码是重新整理的一个文档，所以用第一个短评页面做测试

In [37]:
header = {'Accept': '*/*',
        'Accept-Language': 'en-US,en;q=0.8',
        'Cache-Control': 'max-age=0',
        'User-Agent': 'Mozilla/5.0 (Windows NT 6.2; WOW64; rv:21.0) Gecko/20100101 Firefox/21.0',
        'Connection': 'keep-alive'}
url_count = 0 # 计数：爬取页面的数目(一共有1000个短评首页)
for comment_url in comment_urls[:1]:
    
    # 用伪造的cookies值构造cookies
    jar = requests.cookies.RequestsCookieJar()
    # 随机从伪造的cookies值里抽取一个值
    jar.set(random.choice(cookies)[0], random.choice(cookies)[1], domain='.douban.com', path='/')
    
    # 取出每部电影的地址的主要部分
    main = comment_url.split('?', 1)[0]
    url_count += 1
    
    # 因为是爬取的免费代理IP，所获取的IP虽然经过验证是有效的，但是不一定是有效的
    # 因此请求出现异常有可能是由于无效的IP地址引起的，此时换一个IP，并将这个IP从列表中去掉
    try:
        proxy = random.choice(proxies) # 随机抽取一个代理
        # 请求页面
        r = s.get(next_page, headers=header, 
                proxies=proxy, 
                cookies=jar,
                verify=False)
        r.encoding = 'utf-8' # 设置编码
        # 如果页面中出现这两个信息说明被封号了，需要重新登陆，这时退出循环
        if (('帐号被暂时锁定' in r.text) or ('检测到有异常请求从你的 IP 发出' in r.text)):
            print ('被封号啦')
            print (r.url)
            break
        page = etree.HTML(r.text)
        # 解析页面内容，并写入文件
        comments = get_comments(page) 
        with open('comments.csv', 'a', encoding='utf-8') as f:
            for comment in comments:
                f.write(comment)
                f.write('\n')
        print ('抓取成功：', r.url)
        time.sleep(3)
    except:
        proxies.remove(proxy) # 将列表中的无效IP去掉
        proxy = random.choice(proxies) # 随机抽取一个IP
        r = s.get(next_page, headers=header, 
                    proxies=proxy, 
                    cookies=jar,
                    verify=False)
        r.encoding = 'utf-8'
        if (('帐号被暂时锁定' in r.text) or ('检测到有异常请求从你的 IP 发出' in r.text)):
            print ('被封号啦')
            print (r.url)
            break
        page = etree.HTML(r.text)
        comments = get_comments(page)
        with open('comments.csv', 'a', encoding='utf-8') as f:
            for comment in comments:
                f.write(comment)
                f.write('\n')
        print ('抓取成功：', r.url)
        time.sleep(3)
        continue
        
    # 如果页面中有下一页，则继续抓取下一页
    while page.xpath('//a[@class="next"]/@href'):
        next_page = main + page.xpath('//a[@class="next"]/@href')[0] # 构建下一页的地址
        # 重复上面的页面抓取和解析过程
        try:
            proxy = random.choice(proxies)
            r = s.get(next_page, headers=header, 
                      proxies=proxy, 
                      cookies=jar,
                      verify=False)
            r.encoding = 'utf-8'
            if (('帐号被暂时锁定' in r.text) or ('检测到有异常请求从你的 IP 发出' in r.text)):
                print ('被封号啦')
                print (r.url)
                break
            page = etree.HTML(r.text)
            comments = get_comments(page)
            with open('comments.csv', 'a', encoding='utf-8') as f:
                for comment in comments:
                    f.write(comment)
                    f.write('\n')
            print ('抓取成功：', r.url)
            time.sleep(3)
        except:
            proxies.remove(proxy)
            proxy = random.choice(proxies)
            r = s.get(next_page, headers=header, 
                      proxies=proxy, 
                      cookies=jar,
                      verify=False)
            r.encoding = 'utf-8'
            if (('帐号被暂时锁定' in r.text) or ('检测到有异常请求从你的 IP 发出' in r.text)):
                print ('被封号啦')
                print (r.url)
                break
            page = etree.HTML(r.text)
            comments = get_comments(page)
            with open('comments.csv', 'a', encoding='utf-8') as f:
                for comment in comments:
                    f.write(comment)
                    f.write('\n')
            print ('抓取成功：', r.url)
            time.sleep(3)
            continue
    else:
        print ('没有权限访问:', r.url, url_count)

抓取成功： https://movie.douban.com/subject/1291546/comments?status=P&percent_type=h
抓取成功： https://movie.douban.com/subject/1291546/comments?start=20&limit=20&sort=new_score&status=P&percent_type=h
抓取成功： https://movie.douban.com/subject/1291546/comments?start=40&limit=20&sort=new_score&status=P&percent_type=h
抓取成功： https://movie.douban.com/subject/1291546/comments?start=60&limit=20&sort=new_score&status=P&percent_type=h
抓取成功： https://movie.douban.com/subject/1291546/comments?start=80&limit=20&sort=new_score&status=P&percent_type=h
抓取成功： https://movie.douban.com/subject/1291546/comments?start=100&limit=20&sort=new_score&status=P&percent_type=h
抓取成功： https://movie.douban.com/subject/1291546/comments?start=120&limit=20&sort=new_score&status=P&percent_type=h
抓取成功： https://movie.douban.com/subject/1291546/comments?start=140&limit=20&sort=new_score&status=P&percent_type=h
抓取成功： https://movie.douban.com/subject/1291546/comments?start=160&limit=20&sort=new_score&status=P&percent_type=h
抓取成功： https: